In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Collecting

In [ ]:
pip install dtw-python

In [ ]:
# Libraries
import pandas as pd
import numpy as np
#from numpy import genfromtxt
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
from dtw import *

# Reading data - fetching TRAIN.csv data from the long version

In [ ]:
#lenth_of_line = 6695 # can be automated
df_train_tmp = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TRAIN_LONG.CSV", error_bad_lines=False, warn_bad_lines=False)
lenth_of_line = df_train_tmp.shape[1]
print ("lenth_of_line", lenth_of_line)

colnames = list()
colnames.append("id")
colnames.append("game_type")
for i in range (lenth_of_line-2):
    colnames.append("move"+str(i))
len (colnames)

df_train = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TRAIN_LONG.CSV",names=colnames, header=None, error_bad_lines=False, warn_bad_lines=False)
n_training_games = df_train.shape[0]
print ("n_training_games", n_training_games)

In [ ]:
df_train.head(3)

# Cleaning Training Data

In [ ]:
# Cleaning Training Data by removing all games with no move at all (false starts most likely)
#df_train.dropna(how='all')
df_train.dropna(subset=['move0'], inplace=True)
df_train.reset_index(drop=True, inplace = True)
df_train.tail(3)

# Reading data - fetching TEST.csv data

In [ ]:
# Reading data - fetching TEST.csv data

#lenth_of_line = 3446 # can be automated
df_train_tmp = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TEST_LONG.CSV", error_bad_lines=False, warn_bad_lines=False)
lenth_of_line = df_train_tmp.shape[1]
print ("lenth_of_line", lenth_of_line)
colnames = list()
colnames.append("game_type")
for i in range (lenth_of_line-1):
    colnames.append("move"+str(i))
len (colnames)

df_test = pd.read_csv("/kaggle/input/starcraft-2-player-prediction-challenge-2020/TEST_LONG.CSV",names=colnames, header=None, error_bad_lines=False, warn_bad_lines=False)
n_testing_games = df_test.shape[0]
print ("n_testing_games", n_testing_games)

In [ ]:
df_test.tail(3)

# Extracting Time Series
- To be able to compare time series together we need to give them the same time interval between values.
- To do so we fill empty values with -1 as a filler.
- It is much more difficult to compute the difference between two time series if they have irregular time intervals.
- A problem with this encoding is that hotkey 9 is farther to 1 that 0 is to 1 etc. (Catergical values are being compared)

In [ ]:
# imports
from tqdm import tqdm

> ## Generating time series with only the hot key number

In [ ]:
#action = df.iloc[game_index]["move"+str(move_number)]
# 

def gererate_time_series(df, ts_size = 960):
    #print ("herrre")
    n_games,len_line= df.shape
    #print (n_games,len_line)
    time_series = []
    for index,data in tqdm(df.iterrows()):
        #print ("here0")
        #time_serie = np.zeros(ts_size)
        time_serie = [-1]*ts_size
        for move_number in range(len_line-1):
            move = data["move"+str(move_number)]
            if move == move:
                #print ("here1")
                #print (data["move"+str(move_number)])
                move_split = move.split(":")
                if len(move_split)>=2:
                    #print("here2")
                    #print(data_split[0])
                    #print (data_split)
                    frame = int(move_split[0])
                    event = move_split[1]
                    if frame < ts_size:
                        #print ("here3") 
                        if event == "ControlGroupEvent":
                            event_hotkey = int(move_split[2])
                            event_update_type = move_split[3]
                            time_serie[frame] = event_hotkey
                            #print(event_hotkey)
                            #print (frame)
                            #print(move)
                    else:
                        break
        #print("-------------------------------")
        time_series.append(time_serie) 
    return time_series

In [ ]:
train_time_series = gererate_time_series(df_train.drop(columns = ["id"]),960)

In [ ]:
test_time_series = gererate_time_series(df_test,960)

## Generating time series with only the hot key number
Encoding:
- Ex: 12 -> 1 + 0.2*2 -> 1.4
- Ex: 30 -> 3 + 0.2*0 -> 3.0
- Ex: 91 -> 9 + 0.2*1 -> 9.1

In [ ]:
## Generating time series with only the hot key number

def gererate_time_series_2(df, ts_size = 960):
    #print ("herrre")
    n_games,len_line= df.shape
    #print (n_games,len_line)
    time_series = []
    for index,data in tqdm(df.iterrows()):
        #print ("here0")
        #time_serie = np.zeros(ts_size)
        time_serie = [-1]*ts_size
        for move_number in range(len_line-1):
            move = data["move"+str(move_number)]
            if move == move:
                #print ("here1")
                #print (data["move"+str(move_number)])
                move_split = move.split(":")
                if len(move_split)>=2:
                    #print("here2")
                    #print(data_split[0])
                    #print (data_split)
                    frame = int(move_split[0])
                    event = move_split[1]
                    if frame < ts_size:
                        #print ("here3") 
                        if event == "ControlGroupEvent":
                            event_hotkey = int(move_split[2])
                            event_update_type = move_split[3]
                            time_serie[frame] = float(event_hotkey) + float(event_update_type)*0.2
                            #print(event_hotkey)
                            #print (frame)
                            #print(move)
                    else:
                        break
        #print("-------------------------------")
        time_series.append(time_serie) 
    return time_series 

In [ ]:
train_time_series = gererate_time_series_2(df_train.drop(columns = ["id"]),960)

In [ ]:
test_time_series = gererate_time_series_2(df_test,960)

In [ ]:
# Train
print("We have", len(train_time_series),"time series in the training data")
print("Each time series is set to be of length:",len(train_time_series[0]),"frames (=1min)") # 16frames/sec
print ("Ex",train_time_series[0][680:700])

In [ ]:
# Test
print("We have", len(test_time_series),"time series in the training data")
print("Each time series is set to be of length:",len(test_time_series[0]),"frames (=1min)") # 16frames/sec
print ("Ex",test_time_series[0][680:700])

# Data Visualisation

In [ ]:
from matplotlib import pyplot as plt

## Plot the first n games of the training set

In [ ]:
# Plot the first n games
first_games = 10
plt.figure(1,figsize=(14,5))
#player_id = str(df_train['id'][1])
x = range(1,len(train_time_series[0])+1)
ids = list()
for index, data in df_train.iterrows():
    if len(ids)>first_games:
        break
    #print(type(data['id']))
    #print(ids)
    if str(data['id']) not in ids: 
        plt.scatter(x,train_time_series[index])
        ids.append(data['id'])
    
    
    
plt.title("Games for first "+ str(first_games)+" players")
plt.xlabel('frames')
plt.ylabel('moves')
print ("We see that different games have different signature time series")

## Plot all of the games for a given player

In [ ]:
# Plot all of the games for a given player
plt.figure(1,figsize=(14,5))
player_id = str(df_train['id'][50])
x = range(1,len(train_time_series[0])+1)
for index, data in df_train.head(len(train_time_series)).iterrows():
    if data['id'] == player_id:
        plt.scatter(x,train_time_series[index])

plt.title("All games for player "+ player_id)
plt.xlabel('frames')
plt.ylabel('moves')
print ("Here we see that for a player, The signature similar between games")

# Training KNN

In [ ]:
#custom metric
def DTW(a, b):   
    return dtw(a, b, dist_method='euclidean',keep_internals=True).distance

In [ ]:
import numpy as np
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
X = train_time_series[:1000]
y = list(df_train['id'])[:len(X)]
print (len(X),len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#Train
knn_model = KNeighborsClassifier(metric=DTW, n_neighbors = 1)

## Train with GridSearch
#parameters = {'n_neighbors':[2, 4, 8]}
#knn_model = GridSearchCV(KNeighborsClassifier(metric=DTW), parameters, cv=3, verbose=1)

# fit
print ("start fit")
knn_model.fit(X_train, y_train)

#evaluate
print ("start eval")
y_pred = knn_model.predict(X_test)
print(classification_report(y_test, y_pred))
#print (knn_model.best_estimator_)
#print (knn_model.best_score_)

## Evaluating the unlabeled testing set - Creating submission file

In [ ]:
#Train
knn_model = KNeighborsClassifier(metric=DTW, n_neighbors = 1)
print ("fitting model")
knn_model.fit(train_time_series, list(df_train["id"]))
print ("predicting model")
y_pred = knn_model.predict(test_time_series)

In [ ]:
file_name = "submission7"
df_submission = pd.DataFrame()
df_submission['RowId'] = range(1,len(y_pred)+1)
df_submission['prediction'] = y_pred
df_submission.to_csv('/kaggle/working/'+file_name+'.csv', index = False)
df_submission.head()

# ---------------------MORE VISUALISATION----------------------- 

In [ ]:
#count number of players/labels in training dataset

list_unique_id = np.array(df_train["id"].unique())
print ("there are %d player labels in the training dataset"%len(list_unique_id))

In [ ]:
# [old Code]

# Adding first feature = type of game played
 
# For each id count how many game types he is associated to.
number_players = len(list_unique_id)
game_types = list()
for player_id in list_unique_id:
    
    player_history = df_train[df_train["id"]== player_id]
    game_types.append(len (player_history["game_type"].unique()))
avg = sum(game_types)/number_players
print("average number of types of games played by a player = " +  str(avg))
print(game_types)
print("This shows that a player only plays on one type of game")

In [ ]:
# get information on the length of games

moves_sec = list()
#df_train = df_train.head()

data = df_train.head(1000)
total_moves_list = []
game_times = []
lenth_of_line = df_train.shape[1]
for player_game_index in range (data.shape[0]):
    total_moves = 0
    game_time = 0
    for move_number in range (lenth_of_line-2):
        move = data.iloc[player_game_index]["move"+str(move_number)] #
        #print (move)
        if (not pd.isnull(move)):
            if move[:1] == "t":
                #game_time = int(move[1:])
                game_time += 5 
            else:
                total_moves +=1
    game_time += 0
    game_times.append(game_time)
    total_moves_list.append(total_moves)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1)
plt.title("hist of game time")
plt.hist(game_times, bins=20)
plt.figure(2)
plt.title("hist of number of moves per game")
plt.hist(total_moves_list, bins=20)
#game_times
limit =50
short_games = [time for time in game_times if time<limit]
print ("Only", len(short_games), "games finish before",limit,"seconds")

In [ ]:
# RACE REPARTITION
races = df_train['game_type'].value_counts()
print(races)
plt = races.plot.bar()

In [ ]:
# number of games per player 
player_counts = df_train['id'].value_counts()
print(player_counts.describe())

# list of unique actions
def find_unique_actions(df):
    unique_actions = set()
    actions = df.shape[0]
    for game_index in range(actions):
        for a in df.iloc[game_index]:
            if not pd.isnull(a) and a[0]!='t':
                unique_actions.add(a)
    return unique_actions

unique_hotkeys = sorted(list(find_unique_actions(df_train.drop(['id','game_type'], inplace=False, axis=1))))
print(unique_hotkeys)

keys = list(set([hotkey[:len(hotkey)-1] for hotkey in unique_hotkeys if 'hotkey' in hotkey]))
print(keys)

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    #print("Accuracy:",accuracy)
    return accuracy

# DTW Tutorial

# https://dynamictimewarping.github.io/python/
# https://dynamictimewarping.github.io/py-api/html/
# https://stats.stackexchange.com/questions/109343/dynamic-time-warping-for-irregular-time-series
from dtw import *
#?dtw
#help(DTW)

import numpy as np

## A noisy sine wave as query
idx = np.linspace(0,6.28,num=100)
query = np.sin(idx) + np.random.uniform(size=100)/10.0

## A cosine is for template; sin and cos are offset by 25 samples
template = np.cos(idx)

## Find the best match with the canonical recursion formula
from dtw import *
alignment = dtw(query, template, keep_internals=True)

## Display the warping curve, i.e. the alignment curve
alignment.plot(type="threeway")

## Align and plot with the Rabiner-Juang type VI-c unsmoothed recursion
dtw(query, template, keep_internals=True, 
    step_pattern=rabinerJuangStepPattern(6, "c"))\
    .plot(type="twoway",offset=-2)

## See the recursion relation, as formula and diagram
print(rabinerJuangStepPattern(6,"c"))
rabinerJuangStepPattern(6,"c").plot()

## And much more!